## ETL Data pipeline

In [1]:
# import dependencies 
import requests 
import pandas as pd
import time 
from datetime import datetime
import numpy as np 
import config 

### Extraction 

In [18]:
url = "https://data.austintexas.gov/resource/fdj4-gpfu.json?$limit=100&$where=occ_date between '2018-01-01T00:00:00.000' and '2020-12-31T00:00:00.000'"

In [24]:
# optional parameter for aditonal api parameters/filters
def extract_data(api_endpoint,parameters=None): 
    data_request = requests.get(url=api_endpoint, params=parameters)
    
    data_df = pd.DataFrame.from_records(data_request.json())
    
    return data_df

In [25]:
crime_df = extract_data(url)

In [27]:
crime_df.head(2)

,incident_report_number,crime_type,ucr_code,family_violence,occ_date_time,occ_date,occ_time,rep_date_time,rep_date,rep_time,...,x_coordinate,y_coordinate,latitude,longitude,location,census_tract,clearance_status,clearance_date,ucr_category,category_description
0,201811444,FAMILY DISTURBANCE,3400,N,2018-01-01T16:58:00.000,2018-01-01T00:00:00.000,1658,2018-01-01T16:58:00.000,2018-01-01T00:00:00.000,1658,...,3093814,3093814,30.19737037,-97.80960579,"{'latitude': '30.19737037', 'longitude': '-97....",NaN,NaN,NaN,NaN,NaN
1,20195024716,CRUELTY TO ANIMALS,2717,N,2018-01-01T14:00:00.000,2018-01-01T00:00:00.000,1400,2019-06-17T10:20:00.000,2019-06-17T00:00:00.000,1020,...,3103707,3103707,30.14716858,-97.77960983,"{'latitude': '30.14716858', 'longitude': '-97....",24.28,N,2019-06-18T00:00:00.000,NaN,NaN


In [38]:
[i for i in crime_df.columns]

['incident_report_number',
 'crime_type',
 'ucr_code',
 'family_violence',
 'occ_date_time',
 'occ_date',
 'occ_time',
 'rep_date_time',
 'rep_date',
 'rep_time',
 'location_type',
 'address',
 'zip_code',
 'council_district',
 'sector',
 'district',
 'pra',
 'x_coordinate',
 'y_coordinate',
 'latitude',
 'longitude',
 'location',
 'census_tract',
 'clearance_status',
 'clearance_date',
 'ucr_category',
 'category_description']

### Transform   

In [ ]:
def clean_data(data_df):
    
    #drop columns 
    drop_column = ['latitude', 'longitude', 'location',
                    'occ_date', 'occ_time','rep_date','rep_time','category_description', 'address',
                    'ucr_category', ':@computed_region_a3it_2a2z', ':@computed_region_8spj_utxs',
                    ':@computed_region_q9nd_rr82', ':@computed_region_qwte_z96m']
    
    data_df.drop(drop_column, axis=1, inplace=True)
    
    #rename columns 
    column_names = {'ucr_code':'offense_code','occ_date_time':'ocurred_date',
                    'rep_date_time':'reported_date','crime_type':'offense_type'}
    
    data_df.rename(columns = column_names)
    
    #drop null values 
    
    
    return cleaned_data_df

In [ ]:
def create_tables(cleaned_data_df):
     
    #offense_type_table 
    offense_df = cleaned_data_df[['offense_code','offense_type']].copy()
    offense_df.drop_duplicates(subset='offense_code',inplace=True)
    
    
    #create incident_location_table 
    location_df = cleaned_data_df['location_type'].copy()
    location_df.to_frame()
    location_df.drop_duplicates(inplace=True)
    location_df['location_code'] = np.arrange(len(incident_loc_df))
    
    #create mapping for location_code column
    loc_mapping_df = location_df.copy()
    loc_mapping_df.set_index('location_type', inplace=True)
    location_mapper = loc_mapping_df.to_dict()['location_code']
    
    # rearrange location_df
    location_df = lcoation_df[['locations_code','location_type']]
    
    
    #crime_incidents_table 
    #create encoded location_code column
    crime_incident_df = cleaned_data_df.copy()
    crime_incident_df['location_code'] = crime_incident_df['location_type'] \
    .apply(lambda x: location_mapper[x])
    
    #crate encoded offense_code colum
    crime_incident_df['offense_code']
    
    #drop columns 
    
    
    return crime_incident_df, location_df, offense_df 

